In [5]:
import json

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')

features = tokenizer(['How many people live in Berlin?', 'How many people live in Berlin?'], ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

tensor([[  9.4506],
        [-11.1605]])


In [4]:
scores

array([-10.693045, -10.743122, -10.722447], dtype=float32)

In [6]:
s = open("../prompt_engineering/self-ask/GPT-3_semantic_search.json")
semantic_search_data = json.load(s)

In [19]:
questions = []
answers = []
for i in range(len(semantic_search_data)):
    q = semantic_search_data[i]['GPT-3-Semantic-Search-Generations']['question']
    a = semantic_search_data[i]['GPT-3-Semantic-Search-Generations']['answer']
    if q == '' or q == ' ':
        pass
    else:
        questions.append(q.strip())
    if a == '' or a == ' ':
        pass
    else:
        answers.append(a.strip())

In [38]:
def scorer(question, answers):
    model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')
    tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')

    features = tokenizer([question] * len(answers), answers,  padding=True, truncation=True, return_tensors="pt")

    model.eval()
    with torch.no_grad():
        scores = model(**features).logits
    return scores


In [39]:
import random 
all_scores = []
for data in semantic_search_data:
    scorer_data = {}
    question = data['GPT-3-Semantic-Search-Generations']['question']
    correct = data['GPT-3-Semantic-Search-Generations']['answer']
    answer = [correct, random.choice(answers), random.choice(answers)]
    scorer_data['question'] = question
    scorer_data['answers'] = answer
    scorer_data['score'] = scorer(question, answer)
    all_scores.append(scorer_data)

In [63]:
temp = all_scores

In [64]:
for i in range(len(all_scores)):
    nums = []
    for num in all_scores[i]['score']:
        nums.append(float(num))
    all_scores[i]['score'] = nums

In [71]:
for i in range(len(all_scores)):
    all_scores[i]['text'] = semantic_search_data[i]['textbook-paragraph']

In [73]:
# with open('cross_encoder_scores.json', 'w', encoding='utf-8') as f:
#     json.dump(all_scores, f, ensure_ascii=False, indent=4)

In [25]:
d = []
for word in ['h', 'j ' , 'sa']:
    tup = ('query', word)
    d.append(tup)

In [38]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('model_name', max_length=512)
scores = model.predict([('Query', 'Paragraph1'), ('Query', 'Paragraph2') , ('Query', 'Paragraph3')])

OSError: model_name is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [41]:
def query_paragraph(query:str, paragraph:list):
    from sentence_transformers import CrossEncoder
    model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)
    tuples = []
    for p in paragraph:
        tup = (query, p)
        tuples.append(tup)
    print(tuples)
    # scores = model.predict(tuples)
    # return scores

In [32]:
import json
s = open("cross_encoder_scores.json")
data = json.load(s)